In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ...?
appl = application_df['APPLICATION_TYPE'].value_counts()
replace_application = list(appl[appl <500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_value_counts = application_df['CLASSIFICATION'].value_counts()
classification_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# Determine which values to replace if counts are less than ..?
replace_class= list(classification_value_counts[classification_value_counts <1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\yatin\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
merged_df = application_df.merge(encode_df, left_index=True, right_index=True).drop(columns=application_cat)
merged_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Split our preprocessed data into our features and target arrays
y = merged_df["IS_SUCCESSFUL"].values
X = merged_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40)

C:\Users\yatin\AppData\Local\Temp/ipykernel_1020/381356203.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = merged_df.drop(["IS_SUCCESSFUL"],1).values


In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Attempt 1: Add Another Layer

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 150)               6600      
                                                                 
 dense_1 (Dense)             (None, 80)                12080     
                                                                 
 dense_2 (Dense)             (None, 30)                2430      
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 21,141
Trainable params: 21,141
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model

# Import needed dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define Checkpoint
os.makedirs("Checkpoints/", exist_ok=True)
checkpoint_path= "checkpoints/weights.{epoch:02d}hdf5"

# Create a callback that saves the model's weights every 5 epochs
cb_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq= 'epoch',
    period=5)

fit_model= nn.fit(X_train, y_train, epochs=100, callbacks=[cb_callback])

Epoch 1/100
804/804 [==============================] - 1s 999us/step - loss: 32612.8594 - accuracy: 0.4931
Epoch 2/100
804/804 [==============================] - 1s 814us/step - loss: 21298.0098 - accuracy: 0.4846
Epoch 3/100
804/804 [==============================] - 1s 779us/step - loss: 12547.8721 - accuracy: 0.4841
Epoch 4/100
804/804 [==============================] - 1s 798us/step - loss: 10286.1240 - accuracy: 0.5094
Epoch 5/100
771/804 [===========================>..] - ETA: 0s - loss: 12.9592 - accuracy: 0.5376
Epoch 5: saving model to checkpoints\weights.05hdf5
804/804 [==============================] - 1s 838us/step - loss: 12.4612 - accuracy: 0.5372
Epoch 6/100
804/804 [==============================] - 1s 792us/step - loss: 0.7122 - accuracy: 0.5362
Epoch 7/100
804/804 [==============================] - 1s 819us/step - loss: 0.6908 - accuracy: 0.5362
Epoch 8/100
804/804 [==============================] - 1s 889us/step - loss: 0.6906 - accuracy: 0.5362
Epoch 9/100
804/804 [

804/804 [==============================] - 1s 789us/step - loss: 0.6907 - accuracy: 0.5362
Epoch 63/100
804/804 [==============================] - 1s 773us/step - loss: 0.6907 - accuracy: 0.5362
Epoch 64/100
804/804 [==============================] - 1s 775us/step - loss: 0.6907 - accuracy: 0.5362
Epoch 65/100
802/804 [============================>.] - ETA: 0s - loss: 0.6907 - accuracy: 0.5363
Epoch 65: saving model to checkpoints\weights.65hdf5
804/804 [==============================] - 1s 787us/step - loss: 0.6907 - accuracy: 0.5362
Epoch 66/100
804/804 [==============================] - 1s 766us/step - loss: 0.6907 - accuracy: 0.5362
Epoch 67/100
804/804 [==============================] - 1s 778us/step - loss: 0.6906 - accuracy: 0.5362
Epoch 68/100
804/804 [==============================] - 1s 762us/step - loss: 0.6908 - accuracy: 0.5362
Epoch 69/100
804/804 [==============================] - 1s 756us/step - loss: 0.6907 - accuracy: 0.5362
Epoch 70/100
801/804 [=====================

In [17]:
# Attempt 2: Change Layer Activation

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 80

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 150)               6600      
                                                                 
 dense_5 (Dense)             (None, 80)                12080     
                                                                 
 dense_6 (Dense)             (None, 1)                 81        
                                                                 
Total params: 18,761
Trainable params: 18,761
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model

# Import needed dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define Checkpoint
os.makedirs("Checkpoints/", exist_ok=True)
checkpoint_path= "checkpoints/weights.{epoch:02d}hdf5"

# Create a callback that saves the model's weights every 5 epochs
cb_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq= 'epoch',
    period=5)

fit_model= nn.fit(X_train, y_train, epochs=100, callbacks=[cb_callback])

Epoch 1/100
804/804 [==============================] - 1s 862us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 2/100
804/804 [==============================] - 1s 794us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 3/100
804/804 [==============================] - 1s 757us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 4/100
804/804 [==============================] - 1s 760us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 5/100
799/804 [============================>.] - ETA: 0s - loss: 8.2639 - accuracy: 0.4643
Epoch 5: saving model to checkpoints\weights.05hdf5
804/804 [==============================] - 1s 803us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 6/100
804/804 [==============================] - 1s 780us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 7/100
804/804 [==============================] - 1s 786us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 8/100
804/804 [==============================] - 1s 782us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 9/100
804/804 [==================

804/804 [==============================] - 1s 800us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 63/100
804/804 [==============================] - 1s 781us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 64/100
804/804 [==============================] - 1s 787us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 65/100
735/804 [==========================>...] - ETA: 0s - loss: 8.2411 - accuracy: 0.4657
Epoch 65: saving model to checkpoints\weights.65hdf5
804/804 [==============================] - 1s 790us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 66/100
804/804 [==============================] - 1s 793us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 67/100
804/804 [==============================] - 1s 762us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 68/100
804/804 [==============================] - 1s 757us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 69/100
804/804 [==============================] - 1s 751us/step - loss: 8.2701 - accuracy: 0.4638
Epoch 70/100
764/804 [=====================

In [23]:
# Attempt 3: Reducing the Number of Epochs

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim= number_input_features, activation= "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation= "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation= "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 80)                3520      
                                                                 
 dense_14 (Dense)            (None, 30)                2430      
                                                                 
 dense_15 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model

# Import needed dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define Checkpoint
os.makedirs("Checkpoints/", exist_ok=True)
checkpoint_path= "checkpoints/weights.{epoch:02d}hdf5"

# Create a callback that saves the model's weights every 5 epochs
cb_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq= 'epoch',
    period=5)

fit_model= nn.fit(X_train, y_train, epochs=40, callbacks=[cb_callback])

Epoch 1/40
804/804 [==============================] - 1s 886us/step - loss: 54803.9180 - accuracy: 0.4803
Epoch 2/40
804/804 [==============================] - 1s 721us/step - loss: 28472.2227 - accuracy: 0.4977
Epoch 3/40
804/804 [==============================] - 1s 722us/step - loss: 54846.9883 - accuracy: 0.4764
Epoch 4/40
804/804 [==============================] - 1s 724us/step - loss: 21275.6992 - accuracy: 0.4932
Epoch 5/40
784/804 [============================>.] - ETA: 0s - loss: 17849.6211 - accuracy: 0.5041
Epoch 5: saving model to checkpoints\weights.05hdf5
804/804 [==============================] - 1s 812us/step - loss: 23940.7090 - accuracy: 0.5034
Epoch 6/40
804/804 [==============================] - 1s 752us/step - loss: 25212.1328 - accuracy: 0.4740
Epoch 7/40
516/804 [==================>...........] - ETA: 0s - loss: 11362.0791 - accuracy: 0.5076